# Setting up

In [1]:
!pip install --quiet --upgrade langchain langchain-community langchain-chroma

In [2]:
! pip install -U langchain langchain-openai

  Using cached regex-2024.9.11-cp312-cp312-win_amd64.whl.metadata (41 kB)
   ---------------------------------------- 0.0/883.8 kB ? eta -:--:--
   --------------------------------------- 883.8/883.8 kB 13.5 MB/s eta 0:00:00
Using cached regex-2024.9.11-cp312-cp312-win_amd64.whl (273 kB)


In [1]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b5505218-e8d4-4721-8081-dd043683e89c-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [5]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"

In [9]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

USER_AGENT environment variable not set, consider setting it to identify your requests.


'Task decomposition is a technique used to break down complex tasks into smaller, more manageable steps. This process allows agents to plan and execute tasks more effectively. Task decomposition can be done using prompting techniques, task-specific instructions, or with human inputs.'

In [10]:
# cleanup
vectorstore.delete_collection()

In [11]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

len(docs[0].page_content)

43131

In [14]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_path = "https://prontera.ru/eu_prime"
    # header_template = None,
    # verify_ssl = True,
    # proxies = None,
    # continue_on_failure = False,
    # autoset_encoding = True,
    # encoding = None,
    # web_paths = (),
    # requests_per_second = 2,
    # default_parser = "html.parser",
    # requests_kwargs = None,
    # raise_for_status = False,
    # bs_get_text_kwargs = None,
    # bs_kwargs = None,
    # session = None,
    # show_progress = True,
)



In [18]:
!pip install sentence-transformers

  Using cached sentence_transformers-3.2.1-py3-none-any.whl.metadata (10 kB)
  Using cached scikit_learn-1.5.2-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached scipy-1.14.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached setuptools-75.3.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached safetensors-0.4.5-cp312-none-win_amd64.whl.metadata (3.9 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
Using cached sentence_transformers-3.2.1-py3-none-any.whl (255 kB)
   ---------------------------------------- 0.0/203.0 MB ? eta -:--:--
   ---------------------------------------- 1.6/203.0 MB 8.4 MB/s eta 0:00:24
    -------------

In [3]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124
     ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
     -------------------- ------------------- 3.1/6.1 MB 15.3 MB/s eta 0:00:01
     ---------------------------------------  6.0/6.1 MB 14.7 MB/s eta 0:00:01
     ---------------------------------------- 6.1/6.1 MB 13.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     --------------------------- ------------ 2.9/4.1 MB 15.2 MB/s eta 0:00:01
     ---------------------------------------- 4.1/4.1 MB 14.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB 15.2 MB/s eta 0:02:45
     ---------------------------------------- 0.0/2.5 GB 15.4 MB/s eta 0:02:43
     ---------------------------------------- 0.0/2.5 GB 15.0 MB/s eta 0:02:47
     ---------------------------------------- 0.0/2.5 GB 14.8 MB/s eta 0:02:49
     ------------

  You can safely remove it manually.


In [28]:
!pip install -U xformers --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124
     ---------------------------------------- 0.0/168.8 MB ? eta -:--:--
      -------------------------------------- 2.9/168.8 MB 13.9 MB/s eta 0:00:12
     - ------------------------------------- 5.8/168.8 MB 14.7 MB/s eta 0:00:12
     -- ------------------------------------ 8.9/168.8 MB 14.6 MB/s eta 0:00:11
     -- ----------------------------------- 12.1/168.8 MB 14.8 MB/s eta 0:00:11
     --- ---------------------------------- 14.9/168.8 MB 14.9 MB/s eta 0:00:11
     ---- --------------------------------- 18.1/168.8 MB 14.8 MB/s eta 0:00:11
     ---- --------------------------------- 21.0/168.8 MB 14.7 MB/s eta 0:00:11
     ----- -------------------------------- 24.1/168.8 MB 14.8 MB/s eta 0:00:10
     ----- -------------------------------- 26.0/168.8 MB 14.2 MB/s eta 0:00:11
     ------ ------------------------------- 29.1/168.8 MB 14.3 MB/s eta 0:00:10
     ------- ------------------------------ 32.2/168.8 MB 14.3 MB/s 

In [50]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("dunzhang/stella_en_400M_v5", trust_remote_code=True).cuda()
# ！The default dimension is 1024, if you need other dimensions, please clone the model and modify `modules.json` to replace `2_Dense_1024` with another dimension, e.g. `2_Dense_256` or `2_Dense_8192` !
# on gpu
# you can also use this model without the features of `use_memory_efficient_attention` and `unpad_inputs`. It can be worked in CPU.
# model = SentenceTransformer(
#     "dunzhang/stella_en_400M_v5",
#     trust_remote_code=True,
#     device="cpu",
#     config_kwargs={"use_memory_efficient_attention": False, "unpad_inputs": False}
# )

c:\Users\mokrota\Documents\GitHub\math_problem_recommender\math_problem_recommender\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
A matching Triton is not available, some optimizations will not be enabled
Traceback (most recent call last):
  File "c:\Users\mokrota\Documents\GitHub\math_problem_recommender\math_problem_recommender\.venv\Lib\site-packages\xformers\__init__.py", line 57, in _is_triton_available
    import triton  # noqa
    ^^^^^^^^^^^^^
ModuleNotFoundError: No module named 'triton'
Some weights of the model checkpoint at dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with anot

In [10]:
# This model supports two prompts: "s2p_query" and "s2s_query" for sentence-to-passage and sentence-to-sentence tasks, respectively.
# They are defined in `config_sentence_transformers.json`
query_prompt_name = "s2p_query"
queries = [
    "print('hello world')",
    "std::cout << \"Hello World!\";",
]

# docs do not need any prompts

docs = [
    "General function in python for printing string output is called print",
    "In c++ to print something out one has to call method std::cout"
]

query_embeddings = model.encode(queries, prompt_name=query_prompt_name)
doc_embeddings = model.encode(docs)
print(query_embeddings.shape, doc_embeddings.shape)

similarities = model.similarity(query_embeddings, doc_embeddings)
print(similarities)
# tensor([[0.8398, 0.2990],
#         [0.3282, 0.8095]])


(2, 1024) (2, 1024)
tensor([[0.5871, 0.5899],
        [0.4614, 0.6874]])


# Scraping latex format of documents

# Splitting scrapped documents by problems

In [11]:
!pip install pylatexenc

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136845 sha256=c522890d842e48e767bfdf9c2ecc2d8249d93e84b39610984f6674ebc1c301b7
  Stored in directory: c:\users\mokrota\appdata\local\pip\cache\wheels\06\3e\78\fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc


In [26]:
import re
from pylatexenc.latexwalker import LatexWalker, LatexEnvironmentNode

def split_latex_by_regex(latex_content, regex_pattern):
    """
    Splits LaTeX content by sections.
    
    Args:
        latex_content (str): The LaTeX document as a string.
    
    Returns:
        dict: A dictionary where each key is a section title, and value is the LaTeX content for that section.
    """
    # Regex to match LaTeX sections
    matches = re.finditer(regex_pattern, latex_content)
    splits = {}
    start_idx = None
    current_section = None

    for match in matches:
        if current_section is not None:
            # Add the previous section
            splits[current_section] = latex_content[start_idx:match.start()].strip()
        current_section = match.group(1)
        start_idx = match.end()
    
    if current_section is not None:
        # Add the last section
        splits[current_section] = latex_content[start_idx:]
    
    return splits

def save_files(files, output_dir="unnamed problems"):
    """
    Saves each LaTeX section to a separate file.
    
    Args:
        sections (dict): Dictionary of sections and their LaTeX content.
        output_dir (str): Directory to save section files.
    """
    import os
    os.makedirs(output_dir, exist_ok=True)

    for file, content in files.items():
        pattern = r"\d+/\d"
        matches = re.findall(pattern, file)
        if matches:
            filename = matches[0].replace('/', '_P') + '.tex'
        else:
            filename = file.replace(' ', '_') + '.tex'

        filename = os.path.join(output_dir, filename)
        with open(filename, "w", encoding="utf-8") as f:
            f.write(content)
        print(f"Saved section '{file}' to {filename}")

if __name__ == "__main__":
    # Example usage
    for i in range(1997, 2002):
        filepath_header = f"IMO-{i}-notes"
        filepath_tail = ".tex"
        with open(filepath_header + filepath_tail, "r", encoding="utf-8") as file:
            latex_content = file.read()

        section_pattern = r"\\section\{(.+?)\}"
        subsection_pattern = r"\\subsection\{(.+?)\}"
        enumerate_item_pattern = r"\\item %% (.+)"

        sections = split_latex_by_regex(latex_content, section_pattern)
        problem_section = sections['Problems']
        problems = split_latex_by_regex(problem_section, enumerate_item_pattern)

        solutions = split_latex_by_regex(latex_content, subsection_pattern)
        save_files(problems, filepath_header), save_files(solutions, filepath_header)


Saved section 'Problem 1' to IMO-1997-notes\Problem_1.tex
Saved section 'Problem 2' to IMO-1997-notes\Problem_2.tex
Saved section 'Problem 3' to IMO-1997-notes\Problem_3.tex
Saved section 'Problem 4' to IMO-1997-notes\Problem_4.tex
Saved section 'Problem 5' to IMO-1997-notes\Problem_5.tex
Saved section 'Problem 6' to IMO-1997-notes\Problem_6.tex
Saved section 'IMO 1997/1' to IMO-1997-notes\1997_P1.tex
Saved section 'IMO 1997/2' to IMO-1997-notes\1997_P2.tex
Saved section 'IMO 1997/3' to IMO-1997-notes\1997_P3.tex
Saved section 'IMO 1997/4' to IMO-1997-notes\1997_P4.tex
Saved section 'IMO 1997/5' to IMO-1997-notes\1997_P5.tex
Saved section 'IMO 1997/6' to IMO-1997-notes\1997_P6.tex
Saved section 'Problem 1' to IMO-1998-notes\Problem_1.tex
Saved section 'Problem 2' to IMO-1998-notes\Problem_2.tex
Saved section 'Problem 3' to IMO-1998-notes\Problem_3.tex
Saved section 'Problem 4' to IMO-1998-notes\Problem_4.tex
Saved section 'Problem 5' to IMO-1998-notes\Problem_5.tex
Saved section 'Probl

In [54]:
def check_similarity(queries, docs):
    query_prompt_name = "s2p_query"

    query_embeddings = model.encode(queries, prompt_name=query_prompt_name)
    doc_embeddings = model.encode(docs)
    print(query_embeddings.shape, doc_embeddings.shape)

    similarities = model.similarity(query_embeddings, doc_embeddings)
    return similarities


In [28]:
!pip install pandas

  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   --------- ------------------------------ 2.6/11.5 MB 13.7 MB/s eta 0:00:01
   -------------------- ------------------- 5.8/11.5 MB 14.7 MB/s eta 0:00:01
   ------------------------------ --------- 8.9/11.5 MB 14.9 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.5 MB 15.0 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 14.1 MB/s eta 0:00:00
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)


In [34]:
import pandas as pd
import os

problems = []
for i in range(1997, 2002):
    entry = {}
    entry['year'] = i
    for j in range(1, 7):
        problem_path = os.path.abspath(f'./IMO-{i}-notes/Problem_{j}.tex')
        with open(problem_path, 'r') as f:
            entry[f'problem_{j}'] = f.read()

        solution_path = os.path.abspath(f"./IMO-{i}-notes/{i}_P{j}.tex")
        with open(solution_path, 'r') as f:
            entry[f'solution_{j}'] = f.read()
    problems.append(entry)
    

In [37]:
df = pd.DataFrame(problems)

In [43]:
df

,year,problem_1,solution_1,problem_2,solution_2,problem_3,solution_3,problem_4,solution_4,problem_5,solution_5,problem_6,solution_6
0,1997,In the plane there is an infinite chessboard.\...,\textsl{Available online at \url{https://aops....,Let $ABC$ be a triangle with $\angle A < \min(...,\textsl{Available online at \url{https://aops....,"Let $x_1$, $x_2$, \dots, $x_n$ be real numbers...",\textsl{Available online at \url{https://aops....,An $n \times n$ matrix whose entries come\nfro...,\textsl{Available online at \url{https://aops....,"Find all pairs $(a,b)$ of positive integers sa...",\textsl{Available online at \url{https://aops....,"\nFor each positive integer $n$,\nlet $f(n)$ d...",\n\textsl{Available online at \url{https://aop...
1,1998,A convex quadrilateral $ABCD$ has perpendicula...,\textsl{Available online at \url{https://aops....,"In a competition, there are $a$ contestants\na...",\textsl{Available online at \url{https://aops....,"For any positive integer $n$,\nlet $\tau(n)$ d...",\textsl{Available online at \url{https://aops....,"Determine all pairs $(x,y)$ of positive intege...",\textsl{Available online at \url{https://aops....,Let $I$ be the incenter of triangle $ABC$.\nLe...,\textsl{Available online at \url{https://aops....,\nClassify all functions $f \colon \NN \to \NN...,\n\textsl{Available online at \url{https://aop...
2,1999,A set $S$ of points from the space will be cal...,\textsl{Available online at \url{https://aops....,Find the least constant $C$ such that for any ...,\textsl{Available online at \url{https://aops....,Let $n$ be an even positive integer.\nFind the...,\textsl{Available online at \url{https://aops....,"Find all pairs of positive integers $(x,p)$\ns...",\textsl{Available online at \url{https://aops....,Two circles $\Omega_{1}$ and $\Omega_{2}$ touc...,\textsl{Available online at \url{https://aops....,\nFind all the functions $f \colon \RR \to \RR...,\n\textsl{Available online at \url{https://aop...
3,2000,Two circles $G_1$ and $G_2$ intersect at two p...,\textsl{Available online at \url{https://aops....,"Let $a$, $b$, $c$ be positive real numbers wit...",\textsl{Available online at \url{https://aops....,Let $n \ge 2$ be a positive integer\nand $\lam...,\textsl{Available online at \url{https://aops....,A magician has one hundred cards numbered $1$ ...,\textsl{Available online at \url{https://aops....,Does there exist a positive integer $n$\nsuch ...,\textsl{Available online at \url{https://aops....,"\nLet $\ol{AH_1}$, $\ol{BH_2}$, and $\ol{CH_3}...",\n\textsl{Available online at \url{https://aop...
4,2001,Let $ABC$ be an acute-angled triangle with $O$...,\textsl{Available online at \url{https://aops....,"Let $a$, $b$, $c$ be positive reals. Prove tha...",\textsl{Available online at \url{https://aops....,Twenty-one girls and twenty-one boys took part...,\textsl{Available online at \url{https://aops....,"Let $n > 1$ be an odd integer and let $c_1$, $...",\textsl{Available online at \url{https://aops....,Let $ABC$ be a triangle.\nLet $\ol{AP}$ bisect...,\textsl{Available online at \url{https://aops....,\nLet $a > b > c > d > 0$ be integers satisfyi...,\n\textsl{Available online at \url{https://aop...


In [47]:
problems = []
for index, row in df.iterrows():
    year = row['year']
    for i in range(1, 7):
        problems.append((str(year) + f'P_{i}', row[f'problem_{i}']))

In [77]:
print(queries)

['In the plane there is an infinite chessboard.\nFor any pair of positive integers $m$ and $n$,\nconsider a right-angled triangle with vertices at lattice points\nand whose legs, of lengths $m$ and $n$, lie along edges of the squares.\nLet $S_1$ be the total area of the black part of the triangle\nand $S_2$ be the total area of the white part.\nLet $f(m,n) = | S_1 - S_2 |$.\n\n\n\\begin{enumerate}[(a)]\n  \\ii Calculate $f(m,n)$ for all positive integers $m$ and $n$\n  which are either both even or both odd.\n  \\ii Prove that $f(m,n) \\leq \\frac 12 \\max \\{m,n\\}$ for all $m$ and $n$.\n  \\ii Show that there is no constant $C$\n  such that $f(m,n) < C$ for all $ m$ and $ n$.\n\\end{enumerate}', 'Let $ABC$ be a triangle with $\\angle A < \\min(\\angle B, \\angle C)$.\nThe points $B$ and $C$ divide the circumcircle of the triangle into two arcs.\nLet $U$ be an interior point of the arc between $B$ and $C$ which does not contain $A$.\nThe perpendicular bisectors of $ AB$ and $ AC$ meet

In [55]:
queries = [i[1] for i in problems]
docs = queries
similarity_tensor = check_similarity(queries, docs)

(30, 1024) (30, 1024)


In [60]:
import torch
score, indeces = torch.topk(similarity_tensor, k=2, dim=1)

In [74]:
other_similar = [(int(el[1]), index, float(score[index][1])) for index, el in enumerate(indeces)]

In [81]:
from random import choice

def random_similar(couples, df):
    similar_index, index, score = choice(couples)
    problem = df[index]
    similar_problem = df[similar_index]
    return problem, similar_problem, score

In [83]:
p, similar_p, score = random_similar(other_similar, queries)
p, similar_p, score

('Does there exist a positive integer $n$\nsuch that $n$ has exactly 2000 distinct prime divisors\nand $n$ divides $2^n + 1$?',
 'For any positive integer $n$,\nlet $\\tau(n)$ denote the number of its positive divisors (including $1$ and itself).\nDetermine all positive integers $m$ for which\nthere exists a positive integer $n$ such that\n\\[ \\frac{\\tau(n^{2})}{\\tau(n)}=m. \\]',
 0.7479963302612305)